# Python calculation to be made for individual models

## 0. Packages and data

In [ ]:
cd /content

/content


In [ ]:
!git clone https://github.com/folsomcp/specpolFlow

fatal: destination path 'specpolFlow' already exists and is not an empty directory.


In [ ]:
import specpolFlow as pol
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/Shareddrives/BeStarsMiMeS/

/content/drive/Shareddrives/BeStarsMiMeS


## 1 Remove lines in the mask that are in regions of heavy telluric contamination - Federico

### Based on the _04-Telluric contamination_ notebook:

In the _04-Telluric contamination_ notebook, a code was written to use only spectral lines which are not in the contaminated regions of the spectrum. These regions were identified visually and put in the spreadsheet _04-ContaminationregionsWL_. 

In this notebook we create a loop over all the individual models, with different mask values to 'clean' them.


The two next code blocks just makes sure we are located correctly and authenticates the user. However we change the directory to the _/Masks_ folder. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/Shareddrives/BeStarsMiMeS/Masks

/content/drive/Shareddrives/BeStarsMiMeS/Masks


The next line creates a new folder called __depth0.02_tellClean__, where the new masks will be stored. 

In [ ]:
#!mkdir depth0.02_tellClean

The next code block gathers information from the _00InputInformation_ and _04-ContaminationregionsWL_ spreadsheets using pandas. 

In [ ]:

gc = gspread.authorize(creds)
# Using pandas to read spreadsheet
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1M6y1Wnsrc-w5FjUMfKaSFa_-foIDAaMe8W4lYNWnWyk/edit?usp=sharing')
wsheet = spreadsheet.worksheet("Stars")

lom = pd.DataFrame(wsheet.get_all_records())

#Regions
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/19lS0Xg-2ZUs0ps8jZ-JM3pR1YIuC_lWvRMpFAM5VUYI/edit?usp=sharing')

# Get the name of all of the sheets in the spreadsheet file
print(spreadsheet.worksheets())

# Select the sheet that we want
worksheet = spreadsheet.worksheet("WLRegions")

# get_all_values gives a list of rows.
rows = worksheet.get_all_records()
print(rows[:])

# Put the sheet containing the target information into a 

WLRegions = pd.DataFrame.from_records(rows)


[<Worksheet 'WLRegions' id:0>]
[{'WLStart': 587.5, 'WLFinish': 592, 'Type': 'Telluric'}, {'WLStart': 627.5, 'WLFinish': 632.5, 'Type': 'Telluric'}, {'WLStart': 684, 'WLFinish': 705.3, 'Type': 'Telluric'}, {'WLStart': 717, 'WLFinish': 735, 'Type': 'Telluric'}, {'WLStart': 757, 'WLFinish': 771, 'Type': 'Telluric'}, {'WLStart': 790, 'WLFinish': 794, 'Type': 'Telluric'}, {'WLStart': 809, 'WLFinish': 990, 'Type': 'Telluric'}, {'WLStart': 360, 'WLFinish': 392, 'Type': 'Hjump'}, {'WLStart': 654.1123333, 'WLFinish': 658.4876667, 'Type': 'Halpha'}, {'WLStart': 484.5195333, 'WLFinish': 487.7604667, 'Type': 'Hbeta'}, {'WLStart': 432.6031667, 'WLFinish': 435.4968333, 'Type': 'Hgamma'}, {'WLStart': 408.8027667, 'WLFinish': 411.5372333, 'Type': 'Hdelta'}, {'WLStart': 395.6866333, 'WLFinish': 398.3333667, 'Type': 'Hepsilon'}]


This last code block uses the code from _04-Telluric contamination_ notebook, but loops for all model masks and finally saves the masks with the same name, but located in a the new folder that was created previously. 

In [ ]:
#Place the regions into a matrix

L = np.shape(WLRegions.loc[:,'WLStart'])[0]

regions = np.zeros((L,2))

regions[:,0] = WLRegions.loc[:,'WLStart']
regions[:,1] = WLRegions.loc[:,'WLFinish']

#Getting masks
for i in range(len(lom["Name"])): 
  star = lom["Name"][i]
  modelcode =lom["ModelCode"][i].upper()
  
  #The next loop loops over every observation, which is needed
  #for Asif's masks, so better leave it this way to change
  #easier between masks although our masks only depende on the
  #modelcode
  for j in range(1,int(lom["Nobs"][i])+1):
    name = '/content/drive/Shareddrives/BeStarsMiMeS/02-Mask-calculations/VALDlist2mask/depth0.1/{}_depth0.1.mask'.format(modelcode)
    #name= 'Asif_tweak/{}_clean_{}.mask_auto_twk'.format(star,j)
    mask_i = pol.iolsd.mask(fname=name)

    #Identifying if a line is inside or outside the regions
    dcard = []
    for lines in mask_i.wl:
      for i in range(0,np.size(regions[:,0])):
        if regions[i,0] < lines < regions[i,1]:
          dcard.append(lines)

    #Creating a list with the mask.wl elements
    mask_list = mask_i.wl.tolist()

    for line in dcard:
      index = (mask_list).index(line)
      mask_i.iuse[index]=0

    l = np.size(mask_i.wl)
    spec_lines = []
    for i in range(0,l):
      if mask_i.iuse[i]==1:
        spec_lines.append(mask_i.wl[i])
    mask_i.save('/content/drive/Shareddrives/BeStarsMiMeS/02-Mask-calculations/MaskCleaning/depth0.1_tellClean/{}_depth0.1.mask'.format(modelcode))
    #mask_i.save('Asif_tweak_tellClean/{}_clean_{}.mask_auto_twk'.format(star,j))